Wave 0: Slope based tests

In [1]:
# Imports
import numpy as np
from river import drift
import statistics
import matplotlib.pyplot as plt

In [2]:
# Parameters
u = 0.1
n = 1000
k = 50
slope = [0.001, 0.002, 0.004, 0.01, 1] 
delta = [0.0005, 0.001, 0.002, 0.005, 0.01]

size = 10

In [3]:
# Generate data for 3 distributions
print("Generating Streams...")
random_state = np.random.RandomState(seed=42)

streams = {}

for s in slope:
    streams[s] = []

    for _i in range(size):

        stream = np.array([])
        width = int(1 / s)

        for i in range(k):
            up = [stream]

            for i in range(width):
                up.append(random_state.normal(0.2+0.1*(i/width), u, 1))

            up.append(random_state.normal(0.3, u, n - width))

            for i in reversed(range(width)):
                up.append(random_state.normal(0.2+0.1*(i/width), u, 1))

            up.append(random_state.normal(0.2, u, n - width))

            stream = np.concatenate(up)
        
        #plt.plot(stream)
        #plt.show()
        streams[s].append(stream)
        
print("Stream Generated")

Generating Streams...
Stream Generated


In [4]:
# Functions

def average(data):
    data = [i for i in data if i!=0]
    mean = round(statistics.mean(data), 1)
    sd = round(statistics.stdev(data), 1)
    out = f"{mean} ± {sd}"
    return out

In [5]:
def test_it(s):
    delta_result = []
    delta_result2 = []

    detection_rate = []
    detection_rate2 = []

    _stream = streams[s]

    print(f"Slope = {s}")

    for o in delta:

        print(f"Delta = {o}")
        
        # Adwin
        drifts = [0] * k * 2  * size
        drift_counts = []

        for s, stream in enumerate(_stream):
            
            drift_detector = drift.ADWIN(delta=o)
            drift_count = 0

            for i, val in enumerate(stream):
                if i % n == 0 :
                    print(f"\rProgress: {i}", end = '\r')
                drift_detector.update(val)   # Data is processed one sample at a time

                if drift_detector.drift_detected:
                    # The drift detector indicates after each sample if there is a drift in the data
                    #print(f'Change detected at index {i} by ADWIN')
                    #drifts.append(i)
                    drift_count += 1
                    current = i // n
                    if drifts[current + (s * k * 2)] == 0:
                        drifts[current + (s * k * 2)] = i % n

            drift_counts.append(drift_count)


        print("ADWIN Complete     ")

        print("Results:")
        print(f"ADWIN Drifts: {average(drift_counts)}")
        print(f"Average Detection Time {average(drifts)}")

        delta_result.append(drifts)
        detection_rate.append(drift_counts)

        # Atwin
        test_drift_detector = drift.atwin.ATWIN(delta=o, temp=1/10)
        
        test_drifts = [0] * k * 2 * size
        test_drift_counts = []

        for s, stream in enumerate(_stream):

            test_drift_count = 0

            for i, val in enumerate(stream):
                if i % 1000 == 0 :
                    print(f"\rProgress: {i}", end = '\r')
                test_drift_detector.update(val)

                if test_drift_detector.drift_detected:
                    test_drift_count += 1
                    current = i // n
                    if test_drifts[current + (s * k * 2)] == 0:
                        test_drifts[current + (s * k * 2)] = i % n
            test_drift_counts.append(test_drift_count)
        print("ATWIN Complete     ")

        print("Results:")
        print(f"ATWIN Drifts: {average(test_drift_counts)}")
        print(f"Average Detection Time {average(test_drifts)}")

        delta_result2.append(test_drifts)
        detection_rate2.append(test_drift_counts)

        """
        plt.plot(stream)
        for _d in _drifts:
            plt.axvline(_d, color='red')
        for _d in _test_drifts:
            plt.axvline(_d, color='blue')
        plt.show()
        """
        
    return delta_result, delta_result2, detection_rate, detection_rate2

In [6]:
slope_result = []
slope_result2 = []
d_rate = []
d_rate2 = []

"""
for s in slope:
    a, b = test_it(s)
    slope_result.append(a)
    slope_result2.append(b)
"""

'\nfor s in slope:\n    a, b = test_it(s)\n    slope_result.append(a)\n    slope_result2.append(b)\n'

In [7]:
s = slope[0]
a, b, c, d = test_it(s)
slope_result.append(a)
slope_result2.append(b)
d_rate.append(c)
d_rate2.append(d)

Slope = 0.001
Delta = 0.0005
ADWIN Complete     
Results:
ADWIN Drifts: 56.4 ± 1.2
Average Detection Time 453.7 ± 287.1
ATWIN Complete     
Results:
ATWIN Drifts: 57.4 ± 1.8
Average Detection Time 467 ± 293.6
Delta = 0.001
ADWIN Complete     
Results:
ADWIN Drifts: 61.7 ± 2.1
Average Detection Time 431 ± 322.9
ATWIN Complete     
Results:
ATWIN Drifts: 62 ± 2.2
Average Detection Time 403.6 ± 316.1
Delta = 0.002
ADWIN Complete     
Results:
ADWIN Drifts: 68 ± 2.2
Average Detection Time 431.9 ± 348.6
ATWIN Complete     
Results:
ATWIN Drifts: 68.5 ± 1.9
Average Detection Time 428.4 ± 350.1
Delta = 0.005
ADWIN Complete     
Results:
ADWIN Drifts: 79.7 ± 2.3
Average Detection Time 440.6 ± 385.4
ATWIN Complete     
Results:
ATWIN Drifts: 80.8 ± 2.3
Average Detection Time 445.3 ± 389.2
Delta = 0.01
ADWIN Complete     
Results:
ADWIN Drifts: 93.8 ± 2.5
Average Detection Time 588.0 ± 407.2
ATWIN Complete     
Results:
ATWIN Drifts: 94.2 ± 2.8
Average Detection Time 591.1 ± 405.6


In [8]:
s = slope[1]
a, b, c, d = test_it(s)
slope_result.append(a)
slope_result2.append(b)
d_rate.append(c)
d_rate2.append(d)

Slope = 0.002
Delta = 0.0005
ADWIN Complete     
Results:
ADWIN Drifts: 99.2 ± 0.4
Average Detection Time 706.7 ± 75.5
ATWIN Complete     
Results:
ATWIN Drifts: 100 ± 0.0
Average Detection Time 709.9 ± 52.5
Delta = 0.001
ADWIN Complete     
Results:
ADWIN Drifts: 99.5 ± 0.5
Average Detection Time 661.6 ± 74.3
ATWIN Complete     
Results:
ATWIN Drifts: 100 ± 0.0
Average Detection Time 661.8 ± 51.2
Delta = 0.002
ADWIN Complete     
Results:
ADWIN Drifts: 99.9 ± 0.3
Average Detection Time 620.5 ± 63.8
ATWIN Complete     
Results:
ATWIN Drifts: 100 ± 0.0
Average Detection Time 615.0 ± 52.1
Delta = 0.005
ADWIN Complete     
Results:
ADWIN Drifts: 99.9 ± 0.3
Average Detection Time 564.1 ± 65.8
ATWIN Complete     
Results:
ATWIN Drifts: 100 ± 0.0
Average Detection Time 558.3 ± 49.9
Delta = 0.01
ADWIN Complete     
Results:
ADWIN Drifts: 100 ± 0.0
Average Detection Time 528.2 ± 64.7
ATWIN Complete     
Results:
ATWIN Drifts: 100 ± 0.0
Average Detection Time 520.5 ± 48.7


In [9]:
s = slope[2]
a, b, c, d = test_it(s)
slope_result.append(a)
slope_result2.append(b)
d_rate.append(c)
d_rate2.append(d)

Slope = 0.004
Delta = 0.0005
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 453.8 ± 52.2
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 453.7 ± 50.7
Delta = 0.001
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 421.3 ± 44.3
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 420.9 ± 43.7
Delta = 0.002
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 396.1 ± 38.6
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 395.1 ± 37.3
Delta = 0.005
ADWIN Complete     
Results:
ADWIN Drifts: 99.1 ± 0.3
Average Detection Time 369.7 ± 38.4
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 368.7 ± 33.6
Delta = 0.01
ADWIN Complete     
Results:
ADWIN Drifts: 99.1 ± 0.3
Average Detection Time 348.6 ± 37.9
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 347.9 ± 32.0


In [10]:
s = slope[3]
a, b, c, d = test_it(s)
slope_result.append(a)
slope_result2.append(b)
d_rate.append(c)
d_rate2.append(d)

Slope = 0.01
Delta = 0.0005
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 344.4 ± 32.9
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 343.2 ± 32.5
Delta = 0.001
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 326.7 ± 31.8
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 324.8 ± 31.6
Delta = 0.002
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 307.6 ± 32.4
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 305.8 ± 32.1
Delta = 0.005
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 280.9 ± 34.9
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 278.9 ± 34.2
Delta = 0.01
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 260.7 ± 34.0
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 258.6 ± 33.8


In [11]:
s = slope[4]
a, b, c, d = test_it(s)
slope_result.append(a)
slope_result2.append(b)
d_rate.append(c)
d_rate2.append(d)

Slope = 1
Delta = 0.0005
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 288.3 ± 34.0
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 286.9 ± 32.5
Delta = 0.001
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 268.6 ± 35.2
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 267.0 ± 33.7
Delta = 0.002
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 244.7 ± 35.8
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 243.9 ± 35.6
Delta = 0.005
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 215.7 ± 34.3
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 214.9 ± 33.8
Delta = 0.01
ADWIN Complete     
Results:
ADWIN Drifts: 99 ± 0.0
Average Detection Time 196.7 ± 32.2
ATWIN Complete     
Results:
ATWIN Drifts: 99.9 ± 0.3
Average Detection Time 195.9 ± 31.7


In [12]:
for s, d , d2 in zip(slope, slope_result, slope_result2):
    print("\multirow{2}{2em}{" + str(s) + "}", end="")
    print(" & ADWIN & " + " & ".join([average(t) for t in d]) + " \\\\")
    print(" & ATWIN & " + " & ".join([average(t) for t in d2]) + " \\\\")
    print("\hline")

\multirow{2}{2em}{0.001} & ADWIN & 453.7 ± 287.1 & 431 ± 322.9 & 431.9 ± 348.6 & 440.6 ± 385.4 & 588.0 ± 407.2 \\
 & ATWIN & 467 ± 293.6 & 403.6 ± 316.1 & 428.4 ± 350.1 & 445.3 ± 389.2 & 591.1 ± 405.6 \\
\hline
\multirow{2}{2em}{0.002} & ADWIN & 706.7 ± 75.5 & 661.6 ± 74.3 & 620.5 ± 63.8 & 564.1 ± 65.8 & 528.2 ± 64.7 \\
 & ATWIN & 709.9 ± 52.5 & 661.8 ± 51.2 & 615.0 ± 52.1 & 558.3 ± 49.9 & 520.5 ± 48.7 \\
\hline
\multirow{2}{2em}{0.004} & ADWIN & 453.8 ± 52.2 & 421.3 ± 44.3 & 396.1 ± 38.6 & 369.7 ± 38.4 & 348.6 ± 37.9 \\
 & ATWIN & 453.7 ± 50.7 & 420.9 ± 43.7 & 395.1 ± 37.3 & 368.7 ± 33.6 & 347.9 ± 32.0 \\
\hline
\multirow{2}{2em}{0.01} & ADWIN & 344.4 ± 32.9 & 326.7 ± 31.8 & 307.6 ± 32.4 & 280.9 ± 34.9 & 260.7 ± 34.0 \\
 & ATWIN & 343.2 ± 32.5 & 324.8 ± 31.6 & 305.8 ± 32.1 & 278.9 ± 34.2 & 258.6 ± 33.8 \\
\hline
\multirow{2}{2em}{1} & ADWIN & 288.3 ± 34.0 & 268.6 ± 35.2 & 244.7 ± 35.8 & 215.7 ± 34.3 & 196.7 ± 32.2 \\
 & ATWIN & 286.9 ± 32.5 & 267.0 ± 33.7 & 243.9 ± 35.6 & 214.9 ± 33.8

In [13]:
for s, r, r2 in zip(slope, d_rate, d_rate2):
    print("\multirow{2}{2em}{" + str(s) + "}", end="")

    temp = [str(average([round(100 * (t / (2 * k)), 0) for t in _r])) for _r in r]
    print(" & ADWIN & " + " & ".join(temp) + " \\\\")

    temp2 = [str(average([round(100 * (t / (2 * k)), 0) for t in _r])) for _r in r2]
    print(" & ATWIN & " + " & ".join(temp2) + " \\\\")
    print("\hline")

\multirow{2}{2em}{0.001} & ADWIN & 56.4 ± 1.2 & 61.7 ± 2.1 & 68.0 ± 2.2 & 79.7 ± 2.3 & 93.8 ± 2.5 \\
 & ATWIN & 57.4 ± 1.8 & 62.0 ± 2.2 & 68.5 ± 1.9 & 80.8 ± 2.3 & 94.2 ± 2.8 \\
\hline
\multirow{2}{2em}{0.002} & ADWIN & 99.2 ± 0.4 & 99.5 ± 0.5 & 99.9 ± 0.3 & 99.9 ± 0.3 & 100.0 ± 0.0 \\
 & ATWIN & 100.0 ± 0.0 & 100.0 ± 0.0 & 100.0 ± 0.0 & 100.0 ± 0.0 & 100.0 ± 0.0 \\
\hline
\multirow{2}{2em}{0.004} & ADWIN & 99.0 ± 0.0 & 99.0 ± 0.0 & 99.0 ± 0.0 & 99.1 ± 0.3 & 99.1 ± 0.3 \\
 & ATWIN & 99.9 ± 0.3 & 99.9 ± 0.3 & 99.9 ± 0.3 & 99.9 ± 0.3 & 99.9 ± 0.3 \\
\hline
\multirow{2}{2em}{0.01} & ADWIN & 99.0 ± 0.0 & 99.0 ± 0.0 & 99.0 ± 0.0 & 99.0 ± 0.0 & 99.0 ± 0.0 \\
 & ATWIN & 99.9 ± 0.3 & 99.9 ± 0.3 & 99.9 ± 0.3 & 99.9 ± 0.3 & 99.9 ± 0.3 \\
\hline
\multirow{2}{2em}{1} & ADWIN & 99.0 ± 0.0 & 99.0 ± 0.0 & 99.0 ± 0.0 & 99.0 ± 0.0 & 99.0 ± 0.0 \\
 & ATWIN & 99.9 ± 0.3 & 99.9 ± 0.3 & 99.9 ± 0.3 & 99.9 ± 0.3 & 99.9 ± 0.3 \\
\hline
